# Classification Emergency Case

## Import Package

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import nltk
import os
import re
import string
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

2024-12-02 22:52:27.483111: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 22:52:27.502582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733154747.518818   16575 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733154747.523971   16575 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 22:52:27.544375: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## 1. Preprocessing

### 1. Load Dataset

In [2]:
# Load Dataset
data = pd.read_csv('Dataset/Clean_emergency_case.csv')
print(data.head())

                                                Text Label
0  ada kecelakaan besar di jalan tol banyak korba...  High
1  orang ini pingsan dan tidak bernapas segera ki...  High
2           ada anak kecil tenggelam dia tidak sadar  High
3  kecelakaan sepeda motor dengan luka parah di k...  High
4  seorang pria terjatuh dari gedung tinggi seger...  High


### 2. Normalisasi Teks

In [3]:
# Mengubah teks menjadi huruf kecil
data['Text'] = data['Text'].str.lower()

# Menghapus tanda baca dan karakter non-alfabet
data['Text'] = data['Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

print(data.head())

                                                Text Label
0  ada kecelakaan besar di jalan tol banyak korba...  High
1  orang ini pingsan dan tidak bernapas segera ki...  High
2           ada anak kecil tenggelam dia tidak sadar  High
3  kecelakaan sepeda motor dengan luka parah di k...  High
4  seorang pria terjatuh dari gedung tinggi seger...  High


### 3. Tokenization

In [4]:
nltk.download('punkt_tab')

# Tokenization teks
data['tokens'] = data['Text'].apply(word_tokenize)

print(data[['Text', 'tokens']].head())

                                                Text  \
0  ada kecelakaan besar di jalan tol banyak korba...   
1  orang ini pingsan dan tidak bernapas segera ki...   
2           ada anak kecil tenggelam dia tidak sadar   
3  kecelakaan sepeda motor dengan luka parah di k...   
4  seorang pria terjatuh dari gedung tinggi seger...   

                                              tokens  
0  [ada, kecelakaan, besar, di, jalan, tol, banya...  
1  [orang, ini, pingsan, dan, tidak, bernapas, se...  
2   [ada, anak, kecil, tenggelam, dia, tidak, sadar]  
3  [kecelakaan, sepeda, motor, dengan, luka, para...  
4  [seorang, pria, terjatuh, dari, gedung, tinggi...  


[nltk_data] Downloading package punkt_tab to /home/rafli/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### 4. Stop Words


In [5]:
nltk.download('stopwords')

# Mengambil stopwords Bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

# Menghapus kata-kata tertentu dari daftar stopwords
custom_stopwords = {"tidak", "kecil", "besar"}  # Kata-kata yang ingin dipertahankan
stop_words = stop_words - custom_stopwords  # Menghapus kata-kata tersebut dari stopwords

# Menghapus stopwords
data['tokens'] = data['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

print(data[['Text', 'tokens']].head())


[nltk_data] Downloading package stopwords to /home/rafli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                Text  \
0  ada kecelakaan besar di jalan tol banyak korba...   
1  orang ini pingsan dan tidak bernapas segera ki...   
2           ada anak kecil tenggelam dia tidak sadar   
3  kecelakaan sepeda motor dengan luka parah di k...   
4  seorang pria terjatuh dari gedung tinggi seger...   

                                              tokens  
0   [kecelakaan, besar, jalan, tol, korban, terluka]  
1  [orang, pingsan, tidak, bernapas, kirim, ambul...  
2             [anak, kecil, tenggelam, tidak, sadar]  
3   [kecelakaan, sepeda, motor, luka, parah, kepala]  
4           [pria, terjatuh, gedung, kirim, bantuan]  


### 5. Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Menginisialisasi stemmer Bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Melakukan stemming pada token
data['tokens'] = data['tokens'].apply(lambda tokens: [stemmer.stem(word) for word in tokens])

# Menampilkan hasil
print(data[['Text', 'tokens']].head())

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Membuat tokenizer
tokenizer = Tokenizer()

# Melatih tokenizer pada kolom 'tokens'
tokenizer.fit_on_texts(data['tokens'])

# Mengonversi token menjadi urutan indeks
X = tokenizer.texts_to_sequences(data['tokens'])

# Melakukan padding untuk membuat panjang urutan seragam
X_padded = pad_sequences(X, padding='post', maxlen=100)

print(X_padded.shape)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Mengonversi label menjadi angka
y = label_encoder.fit_transform(data['Label'])

print(y[:5])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['tokens'].apply(lambda tokens: ' '.join(tokens)))

print(X)


In [ ]:
from sklearn.model_selection import train_test_split

# Membagi data menjadi 80% training dan 20% testing
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)

In [ ]:
vocabulary_size = len(tokenizer.word_index) + 1
embedding_dim = 100
max_sequence_length = 10

X_train = tokenizer.texts_to_sequences(data['tokens'])
X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')

print("Vocabulary Size:", vocabulary_size)
print("Sequence Length:", max_sequence_length)
print("X_train (padded sequences):", X_train_padded)

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Conv1D, Bidirectional, GRU, Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model

# Dimensi input untuk teks: panjang urutan (None) dan dimensi vektor kata (misalnya 300 untuk word2vec)
input_text = Input(shape=(None,), name="input_text")

# Embedding Layer untuk merepresentasikan kata-kata dalam bentuk vektor (misalnya pre-trained embedding atau trainable embeddings)
embedding_dim = 300  # Misalnya menggunakan 300 dimensi untuk kata
embedding = Embedding(input_dim=5000, output_dim=embedding_dim, input_length=None)(input_text)  # 5000 adalah ukuran vocabulary

# Normalisasi input embedding
x = LayerNormalization()(embedding)

# Convolutional Layer pertama: 32 filter dengan ukuran kernel (3,), padding 'same' untuk mempertahankan dimensi input, dan fungsi aktivasi ReLU
x = Conv1D(32, kernel_size=3, padding='same', activation='relu')(x)

# Layer normalisasi untuk output dari Conv1D
x = LayerNormalization()(x)

# Bidirectional GRU Layer pertama dengan 128 unit, menghasilkan output berurutan (return_sequences=True)
x = Bidirectional(GRU(128, return_sequences=True))(x)

# Bidirectional GRU Layer kedua dengan 128 unit, menghasilkan output berurutan (return_sequences=True)
x = Bidirectional(GRU(128, return_sequences=True))(x)

# Bidirectional GRU Layer ketiga dengan 128 unit, menghasilkan output berurutan (return_sequences=True)
x = Bidirectional(GRU(128, return_sequences=True))(x)

# Dropout untuk menghindari overfitting
x = Dropout(0.5)(x)

# Dense layer untuk output klasifikasi teks (misalnya untuk klasifikasi multi-kelas)
output = Dense(10, activation="softmax")(x)  # 10 adalah jumlah kelas untuk klasifikasi

# Membuat model dengan input teks dan output hasil prediksi
model = Model(input_text, output, name="Text_Classification")

# Menampilkan ringkasan arsitektur model
model.summary()

In [ ]:
X_train_padded = np.array(X_train_padded)  # Mengonversi ke array NumPy
y_train = np.array(y_train)  # Mengonversi ke array NumPy

X_test_padded = np.array(X_test_padded)  # Mengonversi ke array NumPy
y_test = np.array(y_test)  # Mengonversi ke array NumPy

In [ ]:
model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train, 
    y_train, 
    epochs=5, 
    batch_size=32, 
    validation_split=(X_test_padded, y_test)
)